In [19]:
"""
Module: compute_qubit_freq
--------------------------
Compute the fluxonium qubit frequency E01 using the SQcircuit package.

This method uses SQcircuit's full Hamiltonian diagonalization
and returns the transition frequency between the ground and first
excited states for given circuit parameters.

Author: Jianyao Gu
"""

import SQcircuit as sq
import numpy as np

def compute_qubit_freq(EC, EL, EJ, flux):
    """
    Compute the qubit transition frequency E01 of a fluxonium circuit
    using SQcircuit's diagonalization engine.

    Parameters
    ----------
    EC : float
        Charging energy (in GHz).
    EL : float
        Inductive energy (in GHz).
    EJ : float
        Josephson energy (in GHz).
    flux : float
        External flux in units of magnetic flux quantum Φ₀ (i.e., Φ_ext / Φ₀).

    Returns
    -------
    float
        Transition frequency E01 = E1 - E0 in GHz.
    """

    # Create a flux loop
    loop = sq.Loop(flux)

    # Define circuit elements
    C = sq.Capacitor(EC, "GHz")
    L = sq.Inductor(EL, "GHz", loops=[loop])
    JJ = sq.Junction(EJ, "GHz", cap=C, loops=[loop])

    # Connect elements between node 0 and node 1
    elements = {(0, 1): [L, JJ]}

    # Build the circuit
    circuit = sq.Circuit(elements, flux_dist="all")

    # Truncate Hilbert space and diagonalize
    circuit.set_trunc_nums([60])
    eigenenergy, _ = circuit.diag(2)

    # Return the frequency difference between first two levels
    return eigenenergy[1] - eigenenergy[0]


# Example usage
if __name__ == "__main__":
    EC_val = 3.6  # GHz
    EL_val = 0.46  # GHz
    EJ_val = 10.2  # GHz
    flux_val = 0.0  # Φ_ext / Φ₀

    freq = compute_qubit_freq(EC_val, EL_val, EJ_val, flux_val)
    print(f"Fluxonium qubit transition frequency: {freq:.6f} GHz")

Fluxonium qubit transition frequency: 8.212712 GHz


In [28]:
"""
Module: fluxonium_transition_freq
-------------------------------
Compute the qubit transition frequency (E01) of a fluxonium circuit
using direct numerical diagonalization of the Hamiltonian in the
phase basis.

This implementation uses the finite-difference method to represent
the kinetic term and constructs the Hamiltonian matrix explicitly.
It does *not* rely on external circuit packages (e.g., SQcircuit).
"""

import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import eigsh


def fluxonium_transition_freq(EC, EJ, EL, phi_ext, n_diag=2, phi_max=10 * np.pi, m=2000):
    """
    Compute the lowest two eigenenergies of the fluxonium Hamiltonian
    in the phase basis and return their energy difference (E01).

    Parameters
    ----------
    EC : float
        Charging energy in GHz.
    EJ : float
        Josephson energy in GHz.
    EL : float
        Inductive energy in GHz.
    n_diag: int
        number of eigenvalues and eigenstates we need to diagonalized
    phi_ext : float
        External flux (in radians), equal to 2π * Φ_ext / Φ₀.
    phi_max : float, optional
        Maximum phase extent of the simulation domain.
        Default is 10π (sufficient to include many wells).
    m : int, optional
        Number of discretization points (grid size). Default is 2000.

    Returns
    -------
    float
        Energy difference E01 = E1 - E0 in the same units as EC, EJ, EL
        (typically GHz).

    Notes
    -----
    The Hamiltonian is given by:
        H = 4EC * n² + (1/2)EL(φ - φ_ext)² - EJ * cos(φ)
    with n = -i d/dφ.

    The code uses a finite-difference second derivative
    to approximate the kinetic operator and constructs
    the Hamiltonian matrix in the phase basis.
    """

    # Define the phase grid
    phi = np.linspace(-phi_max, phi_max, m)
    dphi = phi[1] - phi[0]

    # ----- Kinetic term: -4 EC d²/dφ² -----
    main_diag = 2.0 * np.ones(m)
    off_diag = -1.0 * np.ones(m - 1)
    d2 = diags([off_diag, main_diag, off_diag], offsets=[-1, 0, 1]) / (dphi ** 2)
    kinetic = -4.0 * EC * d2

    # ----- Potential term: (1/2)EL(φ - φ_ext)² - EJ cos(φ) -----
    V = 0.5 * EL * (phi - phi_ext) ** 2 - EJ * np.cos(phi)
    potential = diags(V, 0)

    # ----- Total Hamiltonian -----
    H = kinetic + potential

    # ----- Diagonalize to get lowest two eigenenergies -----
    vals, _ = eigsh(H, k=n_diag, which="SA")  # smallest algebraic eigenvalues

    # Return the qubit transition frequency
    return vals[1] - vals[0]


# Example usage
if __name__ == "__main__":
    EC_val = 3.6    # GHz
    EJ_val = 10.2   # GHz
    EL_val = 0.46   # GHz
    phi_ext_val = 2 * np.pi * 0.0  # Φ_ext = 0

    E01 = fluxonium_levels(EC_val, EJ_val, EL_val, phi_ext_val)
    print(f"Fluxonium E01 = {E01:.6f} GHz")

Fluxonium E01 = 8.212746 GHz


In [4]:
"""
Rabi angle for fluxonium under a resonant square pulse (direct numerical method).

We diagonalize the fluxonium Hamiltonian in the phase basis (finite differences),
compute n01 = <0|n̂|1>, and use the RWA two-level result:

    θ = k * V0 * t,   with   k = (2e * η / ħ) * |n01|

where:
- θ is the rotation angle in radians (e.g., π/2 for a π/2 pulse),
- V0 is the peak voltage of the drive at the qubit (Volts),
- t is the pulse duration (seconds),
- η is the voltage division factor (0..1).

All fluxonium energies are in GHz (E/h). Outputs for k are in rad/(V·s).
"""

from __future__ import annotations
import numpy as np
from dataclasses import dataclass
from scipy.sparse import diags
from scipy.sparse.linalg import eigsh

# --- SI constants ---
_E = 1.602176634e-19      # C
_HBAR = 1.054571817e-34   # J*s


@dataclass
class Spectrum:
    energies_ghz: np.ndarray      # shape (M,)
    states: np.ndarray            # shape (grid, M)
    n_op: np.ndarray              # (grid, grid)


# ---------- Fluxonium solver (phase-basis, finite differences) ----------
def solve_fluxonium(EC, EJ, EL, phi_ext, phi_max=10*np.pi, grid=2000, n_levels=4) -> Spectrum:
    """Return (energies, eigenvectors, n-operator) for fluxonium. Energies in GHz."""
    phi = np.linspace(-phi_max, phi_max, grid)
    dphi = phi[1] - phi[0]

    # -4 EC d^2/dphi^2
    main, off = 2.0*np.ones(grid), -1.0*np.ones(grid-1)
    d2 = diags([off, main, off], [-1, 0, 1]) / (dphi**2)
    kinetic = -4.0 * EC * d2

    # Potential: 1/2 EL (phi)^2 - EJ cos(phi-phi_ext)
    V = 0.5*EL*(phi**2) - EJ*np.cos(phi - phi_ext)
    H = kinetic + diags(V, 0)

    vals, vecs = eigsh(H, k=n_levels, which="SA")
    idx = np.argsort(vals)
    E = np.array(vals[idx])
    psi = np.array(vecs[:, idx])

    # n̂ = -i d/dφ (central difference, Dirichlet-like at edges)
    offp =  np.ones(grid-1) / (2.0*dphi)
    offm = -np.ones(grid-1) / (2.0*dphi)
    d1 = diags([offm, np.zeros(grid), offp], [-1, 0, 1]).toarray()
    n_op = (-1j) * d1

    return Spectrum(energies_ghz=E, states=psi, n_op=n_op)


# ---------- Rabi coefficient k, angle, and solvers ----------
def rabi_coefficient(EC, EJ, EL, phi_ext, eta, *, phi_max=10*np.pi, grid=2000) -> dict:
    """
    Compute |n01| and k = (e*eta/ħ) * |n01|  so that  θ = k * V0 * t  (radians).

    Parameters
    ----------
    EC, EJ, EL : float
        Fluxonium energies in GHz (E/h).
    phi_ext : float
        Reduced flux in radians (2π * Φ_ext / Φ0).
    eta : float
        Voltage-division factor (0..1) from drive to qubit island.
    phi_max, grid : discretization controls.

    Returns
    -------
    dict with keys:
        n01_abs : float
            |<0|n̂|1>|.
        k_rad_per_Vs : float
            Rabi coefficient k in rad/(V·s) for peak voltage V0.
        note : str
            Reminder about assumptions (resonant square pulse, RWA).
    """
    spec = solve_fluxonium(EC, EJ, EL, phi_ext, phi_max=phi_max, grid=grid, n_levels=2)
    psi = spec.states
    n_mat = spec.n_op
    n_mn = psi.conj().T @ (n_mat @ psi)    # matrix in eigenbasis
    n01 = n_mn[0, 1]
    k = (2*_E * eta / _HBAR) * np.abs(n01)   # rad/(V·s)  for peak V0

    return {
        "n01_abs": float(np.abs(n01)),
        "k_rad_per_Vs": float(k),
        "note": "θ = k * V0 * t (peak V0), resonant square pulse, RWA.",
    }


def rotation_angle(k_rad_per_Vs: float, V0_volt: float, t_sec: float, *, use_vrms=False) -> float:
    """
    θ = k * V * t. If you pass V_rms, set use_vrms=True (θ increases by √2).
    """
    V_eff = V0_volt if not use_vrms else (np.sqrt(2.0) * V0_volt)
    return k_rad_per_Vs * V_eff * t_sec


def solve_for_time(theta_rad: float, k_rad_per_Vs: float, V0_volt: float, *, use_vrms=False) -> float:
    """t = θ / (k * V). Returns seconds."""
    V_eff = V0_volt if not use_vrms else (np.sqrt(2.0) * V0_volt)
    return theta_rad / (k_rad_per_Vs * V_eff)


def solve_for_amplitude(theta_rad: float, k_rad_per_Vs: float, t_sec: float, *, use_vrms=False) -> float:
    """V = θ / (k * t). Returns Volts; if use_vrms=True, this is V_rms."""
    V_eff = theta_rad / (k_rad_per_Vs * t_sec)
    return V_eff if not use_vrms else (V_eff / np.sqrt(2.0))


# ---------- Convenience for π/2 pulses ----------
def pi_over_2_time(EC, EJ, EL, phi_ext, eta, V0_volt, *, use_vrms=False, phi_max=10*np.pi, grid=2000) -> float:
    """Return t_{π/2} (seconds) for a given peak (or RMS) amplitude."""
    k = rabi_coefficient(EC, EJ, EL, phi_ext, eta, phi_max=phi_max, grid=grid)["k_rad_per_Vs"]
    return solve_for_time(0.5*np.pi, k, V0_volt, use_vrms=use_vrms)


def pi_over_2_amplitude(EC, EJ, EL, phi_ext, eta, t_sec, *, use_vrms=False, phi_max=10*np.pi, grid=2000) -> float:
    """Return V0 (or V_rms) needed for a π/2 rotation in time t_sec."""
    k = rabi_coefficient(EC, EJ, EL, phi_ext, eta, phi_max=phi_max, grid=grid)["k_rad_per_Vs"]
    return solve_for_amplitude(0.5*np.pi, k, t_sec, use_vrms=use_vrms)


# --- Example ---
if __name__ == "__main__":
    EC, EL, EJ = 3.6, 0.46, 10.2   # GHz
    phi_ext = 0.0                  # radians
    eta = 1                     # voltage division |V_qb/V_input|, if directly coupled to fluxonium with a large capacitance eta=1
    out = rabi_coefficient(EC, EJ, EL, phi_ext, eta)
    k = out["k_rad_per_Vs"]
    print(f"|n01| = {out['n01_abs']:.4f},  k = {k:.3e} rad/(V·s)")

    # Example: π/2 duration for a 2 mV_peak pulse
    V0 = 2e-3
    t_pi2 = pi_over_2_time(EC, EJ, EL, phi_ext, eta, V0)
    print(f"t_pi/2 @ {V0*1e3:.2f} mV_peak  ->  {t_pi2*1e9:.6f} ns")

    # Example: amplitude needed for π/2 in 20 ns
    t = 20e-9
    V_need = pi_over_2_amplitude(EC, EJ, EL, phi_ext, eta, t)
    print(f"V0_peak for π/2 in {t*1e9:.2f} ns -> {V_need*1e3:.6f} mV")
#EJ=36.143GHz EC=1.243GHz EL=0.477GHz fr=4.9072GHz g=72MHz (check teams-kou's notebook-fluxonium-simulation)

|n01| = 0.1413,  k = 4.295e+14 rad/(V·s)
t_pi/2 @ 2.00 mV_peak  ->  0.001829 ns
V0_peak for π/2 in 20.00 ns -> 0.000183 mV


In [2]:
import numpy as np
from scipy.sparse import diags, kron, identity, csc_matrix
from scipy.sparse.linalg import eigsh

# ---------- Fluxonium (phase-basis finite differences) ----------
def fluxonium_basis(EC, EL, EJ, phi_ext, levels=8, grid=2000, phi_max=12*np.pi):
    """Return (E[GHz], n_Q projected into eigenbasis) using FD in phase."""
    phi = np.linspace(-phi_max, phi_max, grid)
    dphi = phi[1] - phi[0]

    # Kinetic: -4 EC d^2/dphi^2
    main, off = 2.0*np.ones(grid), -1.0*np.ones(grid-1)
    lap = diags([off, main, off], offsets=[-1, 0, 1]) / (dphi**2)
    T = -4.0 * EC * lap

    # Potential: + 1/2 EL * phi^2 - EJ cos(phi - phi_ext)
    V = 0.5 * EL * (phi**2) - EJ * np.cos(phi - phi_ext)
    H = T + diags(V, 0)

    # Lowest 'levels' eigenpairs
    evals, evecs = eigsh(H, k=levels, which="SA")
    idx = np.argsort(evals)
    E = np.array(evals[idx])                 # GHz
    Psi = np.array(evecs[:, idx])            # grid x levels

    # n̂ = -i d/dphi (central difference; Dirichlet-like edges)
    offp =  np.ones(grid-1) / (2.0 * dphi)
    offm = -np.ones(grid-1) / (2.0 * dphi)
    d1 = diags([offm, np.zeros(grid), offp], [-1, 0, 1], dtype=np.complex128)
    n_grid = (-1j) * d1                       # grid x grid (sparse)

    # Project n̂ into eigenbasis of fluxonium
    nQ = Psi.conj().T @ (n_grid @ Psi)        # levels x levels (dense small)
    return E, nQ

# ---------- Resonator (Fock basis) ----------
def resonator_ops_from_fr(fr_GHz, N_phot=6):
    """Return H_res (GHz), n_R = i(a†-a), N = a†a in N_phot Fock basis."""
    a = np.zeros((N_phot, N_phot), dtype=complex)
    for n in range(1, N_phot):
        a[n-1, n] = np.sqrt(n)
    adag = a.conj().T
    Hres = fr_GHz * (adag @ a + 0.5 * np.eye(N_phot))   # GHz
    nR   = 1j * (adag - a)                               # dimensionless
    Nph  = adag @ a
    return csc_matrix(Hres), csc_matrix(nR), csc_matrix(Nph)

# ---------- Build total Hamiltonian: H = H_Q + H_res + g * n_Q n_R ----------
def build_total_H(EC, EL, EJ, phi_ext, fr_GHz, g_GHz,
                  q_levels=8, N_phot=6, grid=2000, phi_max=12*np.pi):
    # Qubit block
    EQ, nQ = fluxonium_basis(EC, EL, EJ, phi_ext, levels=q_levels, grid=grid, phi_max=phi_max)
    Hq = diags(EQ, 0, format='csc')
    Iq = identity(q_levels, format='csc')

    # Resonator block
    Hres, nR, Nph = resonator_ops_from_fr(fr_GHz, N_phot=N_phot)
    Ir = identity(N_phot, format='csc')

    # Interaction
    Hint = g_GHz * kron(csc_matrix(nQ), nR)

    # Total Hamiltonian
    Htot = kron(Hq, Ir) + kron(Iq, Hres) + Hint
    return Htot, EQ, nQ, Nph, q_levels, N_phot

# ---------- Classify eigenstates and extract f_r(g/e) ----------
def readout_frequencies(EC, EL, EJ, phi_ext, fr_GHz, g_GHz,
                        q_levels=8, N_phot=6, grid=2000, phi_max=12*np.pi, k_eval=None):
    """
    Exact-diagonalization method. Returns:
      f_r_g, f_r_e, two_chi, chi  (all GHz)
    """
    Htot, EQ, nQ, Nph, Lq, Nr = build_total_H(EC, EL, EJ, phi_ext, fr_GHz, g_GHz,
                                              q_levels=q_levels, N_phot=N_phot,
                                              grid=grid, phi_max=phi_max)
    if k_eval is None:
        # a few manifolds around the ground state is enough
        k_eval = min(4*Lq, Htot.shape[0]-2)

    evals, evecs = eigsh(Htot, k=k_eval, which="SA")
    idx = np.argsort(evals)
    E = np.array(evals[idx])            # sorted energies
    V = np.array(evecs[:, idx])         # columns are eigenvectors

    # Projectors onto |g> and |e> (qubit eigenbasis indices 0 and 1)
    Pg = np.zeros((Lq, Lq)); Pg[0, 0] = 1.0
    Pe = np.zeros((Lq, Lq)); Pe[1, 1] = 1.0
    Pg_full = kron(csc_matrix(Pg), identity(Nr, format='csc'))
    Pe_full = kron(csc_matrix(Pe), identity(Nr, format='csc'))
    Nph_full = kron(identity(Lq, format='csc'), Nph)

    # Expectations: <Pg>, <Pe>, <N_ph>
    def expect(op, vec):
        return np.real_if_close((vec.conj().T @ (op @ vec)).item())

    tags = []
    for j in range(E.size):
        v = V[:, j]
        pg = float(expect(Pg_full, v))
        pe = float(expect(Pe_full, v))
        nbar = float(expect(Nph_full, v))
        tags.append((E[j], pg, pe, nbar, j))
    # pick states by manifold & photon number ~0 and ~1
    def pick_state(manifold='g', n_target=0):
        best = None; best_score = 1e9
        for E_j, pg, pe, nbar, j in tags:
            weight = pg if manifold=='g' else pe
            # large weight to be in the right manifold; nbar close to target
            score = (1.0 - weight) + 0.3*abs(nbar - n_target)
            if score < best_score:
                best = (E_j, j, weight, nbar)
                best_score = score
        return best  # (Energy, index, manifold_weight, nbar)

    Eg0, jg0, wg0, ng0 = pick_state('g', 0)
    Eg1, jg1, wg1, ng1 = pick_state('g', 1)
    Ee0, je0, we0, ne0 = pick_state('e', 0)
    Ee1, je1, we1, ne1 = pick_state('e', 1)

    fr_g = Eg1 - Eg0
    fr_e = Ee1 - Ee0
    two_chi = fr_e - fr_g
    chi = 0.5 * two_chi

    return {
        "fr_bare": fr_GHz,
        "fr_g": float(fr_g),
        "fr_e": float(fr_e),
        "two_chi": float(two_chi),
        "chi": float(chi),
        "manifold_info": {
            "g_n0": {"E": float(Eg0), "p_g": float(wg0), "nbar": float(ng0)},
            "g_n1": {"E": float(Eg1), "p_g": float(wg1), "nbar": float(ng1)},
            "e_n0": {"E": float(Ee0), "p_e": float(we0), "nbar": float(ne0)},
            "e_n1": {"E": float(Ee1), "p_e": float(we1), "nbar": float(ne1)},
        },
    }

# ---------- Example ----------
if __name__ == "__main__":
    # Fluxonium (GHz)
    EC, EL, EJ = 1.0, 0.5, 5.3
    phi_ext = 0.0
    # Resonator and coupling (GHz)
    fr = 4.07
    g  = 0.067   # this g multiplies n_Q ⊗ [i(a†-a)] exactly as coded (GHz)

    out = readout_frequencies(EC, EL, EJ, phi_ext, fr, g, q_levels=10, N_phot=8)
    print(f"bare f_r = {out['fr_bare']:.6f} GHz")
    print(f"f_r(g)   = {out['fr_g']:.6f} GHz")
    print(f"f_r(e)   = {out['fr_e']:.6f} GHz")
    print(f"χ       = {out['chi']:.8f} GHz")


bare f_r = 4.070000 GHz
f_r(g)   = 4.068867 GHz
f_r(e)   = 4.067274 GHz
χ       = -0.00079647 GHz
